In [3]:
import numpy as np
import pandas as pd

#load the consolidated dataset
df = pd.read_csv(r'C:\Users\alexm\FX_project\2000_2024\data2.csv')

# remove extra rows and columns
df = df.iloc[200:5383, 0:62]

# reformat Local time variable as a datetime
from datetime import datetime

df['Date'] = df['Date'].str[6:10] +'-'+ df['Date'].str[3:5] +'-'+ df['Date'].str[0:2]
df['Date'] = pd.to_datetime(df['Date'])

df.shape[0]

C:\Users\alexm\AppData\Local\Temp\ipykernel_11376\3067084044.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\alexm\FX_project\2000_2024\data2.csv')


5183

In [4]:
# normalise data

df_norm = df.iloc[:, 4:]
df_dates = df.iloc[:, :4]

for col in df_norm.columns:
    df_norm[col] = (df_norm[col] - df_norm[col].mean())/ df_norm[col].std()
    
x = df_norm.to_numpy()
y = df_dates.to_numpy()
cols = df.columns
z = np.append(y,x, axis=1)

df_norm2 = pd.DataFrame(z, columns=cols)
df_norm2.head()

,Date,Day,Month,Year,Close_Mid,Volume_Bid,Volume_Ask,Volume_Tot,Close_Sprd,5D-MA,...,UK-NomGDP,UK-CPI,UK-Unemp,UK-CorpProf,UK-HshldInc,UK-ResPropInc,UK-SecLend,UK-ConsCred,UK-RetailVol,UK-MktVol
0,2004-02-08,8.0,2.0,2004.0,1.346261,-0.108425,-0.163825,-0.136559,-0.990425,1.269796,...,-1.594999,-1.521179,-0.510423,-1.612364,-1.684136,-1.475755,2.743055,1.654251,1.138495,-0.375961
1,2004-02-09,9.0,2.0,2004.0,1.374563,-0.128922,-0.180773,-0.155302,-0.990425,1.293727,...,-1.594999,-1.521179,-0.510423,-1.612364,-1.684136,-1.475755,2.743055,1.654251,1.138495,-0.375961
2,2004-02-10,10.0,2.0,2004.0,1.471449,-0.131869,-0.179118,-0.155932,-0.684685,1.343293,...,-1.594999,-1.521179,-0.510423,-1.612364,-1.684136,-1.475755,2.743055,1.654251,1.138495,-0.375961
3,2004-02-11,11.0,2.0,2004.0,1.478254,-0.110957,-0.169778,-0.140821,-0.990425,1.391824,...,-1.594999,-1.521179,-0.510423,-1.612364,-1.684136,-1.475755,2.743055,1.654251,1.138495,-0.375961
4,2004-02-12,12.0,2.0,2004.0,1.447614,-0.128374,-0.181841,-0.155568,-0.990425,1.423448,...,-1.594999,-1.521179,-0.510423,-1.612364,-1.684136,-1.475755,2.743055,1.654251,1.138495,-0.375961


In [5]:
# Create features and target datasets

features = df_norm2[['Volume_Tot','Close_Sprd','50D-MA','200D-MA','5D-Vol','5D-ATR','5D-ADX','14D-RSI','US-BaseRate',
                     'UK-BaseRate','UK-5Y-Inf','US-2Y-Int','US-10Y-Int','UK-2Y-Int','UK-10Y-Int','US-RealGDP','US-Unemp',
                     'US-CPI','US-HousePrice','US-CommmRealEst','US-MktVol','UK-RealGDP','UK-CPI','UK-Unemp','UK-CorpProf',
                     'UK-HshldInc','UK-ResPropInc','UK-SecLend','UK-ConsCred','UK-RetailVol','UK-MktVol']]

# we can't use datetime type in the model so we convert to an index
features['Index'] = range(features.shape[0])
target = df_norm2['Close_Mid']

X = features.values
y = target.values

C:\Users\alexm\AppData\Local\Temp\ipykernel_11376\1748117150.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Index'] = range(features.shape[0])


In [20]:
# use the information classifier to select best features
# https://www.rasgoml.com/feature-engineering-tutorials/feature-selection-using-mutual-information-in-scikit-learn

from sklearn.feature_selection import SelectKBest, mutual_info_regression, SelectPercentile
from sklearn.feature_selection import mutual_info_classif

# determine the mutual information
selector = SelectKBest(mutual_info_regression, k=20)
X_reduced = selector.fit_transform(features, target)
X_reduced.shape

(5183, 20)

In [19]:
cols = selector.get_support(indices=True)
selected_columns = features.iloc[:,cols].columns.tolist()
selected_columns

['50D-MA',
 '200D-MA',
 'US-BaseRate',
 'US-RealGDP',
 'US-Unemp',
 'US-CPI',
 'US-HousePrice',
 'US-CommmRealEst',
 'US-MktVol',
 'UK-RealGDP',
 'UK-CPI',
 'UK-Unemp',
 'UK-CorpProf',
 'UK-HshldInc',
 'UK-ResPropInc',
 'UK-SecLend',
 'UK-ConsCred',
 'UK-RetailVol',
 'UK-MktVol',
 'Index']

In [21]:
# create a reduced dataframe
df_reducedFeat = pd.DataFrame(X_reduced, columns=selected_columns)
df_reducedFeat.head()

,50D-MA,200D-MA,US-BaseRate,US-RealGDP,US-Unemp,US-CPI,US-HousePrice,US-CommmRealEst,US-MktVol,UK-RealGDP,UK-CPI,UK-Unemp,UK-CorpProf,UK-HshldInc,UK-ResPropInc,UK-SecLend,UK-ConsCred,UK-RetailVol,UK-MktVol,Index
0,1.062489,0.572248,-0.286315,0.023744,-0.09033,0.29227,-0.844048,-1.41811,-0.457337,-1.574782,-1.521179,-0.510423,-1.612364,-1.684136,-1.475755,2.743055,1.654251,1.138495,-0.375961,0
1,1.074083,0.577747,-0.275069,0.023744,-0.09033,0.29227,-0.844048,-1.41811,-0.457337,-1.574782,-1.521179,-0.510423,-1.612364,-1.684136,-1.475755,2.743055,1.654251,1.138495,-0.375961,1
2,1.08776,0.584124,-0.280692,0.023744,-0.09033,0.29227,-0.844048,-1.41811,-0.457337,-1.574782,-1.521179,-0.510423,-1.612364,-1.684136,-1.475755,2.743055,1.654251,1.138495,-0.375961,2
3,1.1021,0.590424,-0.280692,0.023744,-0.09033,0.29227,-0.844048,-1.41811,-0.457337,-1.574782,-1.521179,-0.510423,-1.612364,-1.684136,-1.475755,2.743055,1.654251,1.138495,-0.375961,3
4,1.115181,0.596511,-0.269445,0.023744,-0.09033,0.29227,-0.844048,-1.41811,-0.457337,-1.574782,-1.521179,-0.510423,-1.612364,-1.684136,-1.475755,2.743055,1.654251,1.138495,-0.375961,4
